# cuGraph Single Source Shortest Path

In this notebook, you will use GPU-accelerated graph analytics with cuGraph to identify the shortest path from node on the road network to every other node, both by distance, which we will demo, and by time, which you will implement. You will also visualize the results of your findings.

## Objectives

By the time you complete this notebook you will be able to:

- Use GPU-accelerated SSSP algorithm
- Use cuXfilter to create a heatmap of average travel times

## Imports

In [1]:
import cudf
import cugraph as cg

import cuxfilter as cxf
from bokeh.palettes import Magma, Turbo256, Plasma256, Viridis256

## Loading Data

We start by loading the road graph data you prepared for constructing a graph with cuGraph, with the long unique `nodeid` replaced with simple (and memory-efficient) integers we call the `graph_id`.

In [3]:
road_graph = cudf.read_csv('./data/road_graph_2-09.csv', dtype=['int32', 'int32', 'float32'])
road_graph.head()

,src,dst,length
0,0,129165,44.0
1,1,1678323,70.0
2,1,2372610,18.0
3,1,2483057,40.0
4,2,2,55.0


Next we load the graph-ready data you prepared that uses amount of time traveled as edge weight.

In [4]:
speed_graph = cudf.read_csv('./data/road_graph_speed_2-09.csv', dtype=['int32', 'int32', 'float32'])
speed_graph.head()

,src,dst,length_s
0,0,129165,3.280848
1,1,1678323,5.219531
2,1,2372610,1.342165
3,1,2483057,2.982589
4,2,2,4.101060


Finally we import the full `road_nodes` dataset, which we will use below for visualizations.

In [5]:
road_nodes = cudf.read_csv('./data/road_nodes_2-09.csv', dtype=['str', 'float32', 'float32', 'str'])
road_nodes = road_nodes.drop_duplicates() # again, some road nodes appeared on multiple map tiles in the original source
road_nodes.head()

,node_id,east,north,type
2589119,id000000F5-5180-4C03-B05D-B01352C54F89,432920.250,572547.375000,road end
1954117,id000003F8-9E09-4829-AD87-6DA4438D22D8,526616.375,189678.390625,junction
873541,id000010DA-C89A-4198-847A-6E62815E038A,336879.000,731824.000000,junction
1346912,id000017A0-1843-4BC7-BCF7-C943B6780839,380635.000,390153.000000,junction
1553458,id00001B2A-155F-4CD3-8E06-7677ADC6AF74,337481.000,350509.687500,junction


In [6]:
road_nodes.shape

(3078117, 4)

In [7]:
speed_graph.src.max()

3078116

## Construct Graph with cuGraph

Now that we have the well-prepped `road_graph` data, we pass it to cuGraph to create our graph datastructure, which we can then use for accelerated analysis. In order to do so, we first use cuGraph to instantiate a `Graph` instance, and then pass the instance edge sources, edge destinations, and edge weights, currently the length of the roads.

In [8]:
G = cg.Graph()
%time G.from_cudf_edgelist(road_graph, source='src', destination='dst', edge_attr='length')

CPU times: user 120 ms, sys: 88 ms, total: 208 ms
Wall time: 209 ms


## Analyzing the Graph

First, we check the number of nodes and edges in our graph:

In [9]:
G.number_of_nodes()

3078117

In [10]:
G.number_of_edges()

3620793

We can also analyze the degrees of our graph nodes. We would expect, as before, that every node would have a degree of 2 or higher, since undirected edges count as two edges (one in, one out) for each of their nodes.

In [11]:
deg_df = G.degree()
deg_df['degree'].describe()[1:]

mean     4.689990
std      1.913452
min      2.000000
25%      2.000000
50%      6.000000
75%      6.000000
max     16.000000
Name: degree, dtype: float64

We would also expect that every degree would be a multiple of 2, for the same reason. We check that there are no nodes with odd degrees (that is, degrees with a value of 1 modulo 2):

In [12]:
deg_df[deg_df['degree'].mod(2) == 1]

,degree,vertex


Observe for reference that some roads loop from a node back to itself:

In [13]:
road_graph.loc[road_graph.src == road_graph.dst]

,src,dst,length
4,2,2,55.0
145,62,62,108.0
293,124,124,67.0
471,196,196,26.0
571,240,240,44.0
...,...,...,...
7216602,3077469,3077469,78.0
7216735,3077519,3077519,111.0
7216849,3077567,3077567,69.0
7217091,3077670,3077670,30.0


## Single Source Shortest Path

To demo the Single Source Shortest Path (SSSP) algorithm, we will start with the node with the highest degree. First we obtain its `graph_id`, reported by the `degree` method as `vertex`:

In [14]:
demo_node = deg_df.nlargest(1, 'degree')
demo_node_graph_id = demo_node['vertex'].iloc[0]
demo_node_graph_id

652907

We can now call `cg.sssp`, passing it the entire graph `G`, and the `graph_id` for our selected vertex. Doing so will calculate the shortest path, using the road length weights we have provided, to *every* other node in the graph - millions of paths, in seconds:

In [15]:
%time shortest_distances_from_demo_node = cg.sssp(G, demo_node_graph_id)
shortest_distances_from_demo_node.head()

CPU times: user 2.21 s, sys: 1.9 s, total: 4.11 s
Wall time: 4.1 s


,distance,vertex,predecessor
0,441163.0,67552,376195
1,119326.0,67553,2650457
2,258332.0,67554,529669
3,272891.0,67555,2123025
4,433889.0,67556,733646


In [16]:
# Limiting to those nodes that were connected (within ~4.3 billion meters because
# cg.sssp uses the max int value for unreachable nodes, such as those on different islands)
shortest_distances_from_demo_node['distance'].loc[shortest_distances_from_demo_node['distance'] < 2**32].describe()[1:]

mean    209942.046612
std     137073.103410
min          0.000000
25%     124952.000000
50%     181649.000000
75%     252309.000000
max     868580.000000
Name: distance, dtype: float64

## Exercise: Analyze a Graph with Time Weights

For this exercise, you are going to analyze the graph of GB's roads, but this time, instead of using raw distance for a road's weights, you will be using how long it will take to travel along the road.

### Step 1: Construct the Graph

Construct a cuGraph graph called `G_ex` using the sources and destinations found in `speed_graph`, along with length in seconds values for the edges' weights.

In [17]:
G_ex = cg.Graph()
%time G_ex.from_cudf_edgelist(speed_graph, source='src', destination='dst', edge_attr='length_s')

CPU times: user 112 ms, sys: 92 ms, total: 204 ms
Wall time: 210 ms


#### Solution

In [18]:
# %load solutions/construct_graph
G_ex = cg.Graph()
G_ex.from_cudf_edgelist(speed_graph, source='src', destination='dst', edge_attr='length_s')


### Step 2: Get Travel Times From a Node to All Others

Choose one of the nodes and calculate the time it would travel to take from it to all other nodes via SSSP, calling the results `ex_dist`.

In [21]:
ex_dist = cg.sssp(G_ex, 0)
ex_dist['distance'].loc[ex_dist['distance'] < 2**32].describe()[1:]

mean    12038.132681
std      5029.636379
min         0.000000
25%      8064.038818
50%     12319.248047
75%     15841.520996
max     26572.105469
Name: distance, dtype: float64

#### Solution

In [20]:
# %load solutions/travel_times
# If you have time, see what the SSSP visualization looks like starting from nodes at different extreme coordinates,
# or one of the end nodes of an especially long edge, or even one of the nodes unreachable from the main road network.
ex_deg = G_ex.degree()
ex_node = ex_deg.nlargest(1, 'degree')

%time ex_dist = cg.sssp(G_ex, ex_node['vertex'].iloc[0])

# limiting to those nodes that were connected (within ~4.3 billion seconds; .sssp uses the max int value for unconnected nodes)
ex_dist['distance'].loc[ex_dist['distance'] < 2**32].describe()[1:]


### Step 3: Visualize the Node Travel Times

In order to create a graphic showing the road network by travel time from the selected node, we first need to align the just-calculated distances with their original nodes. For that, we use the mapping from `node_id` strings to their `graph_id` integers.

In [22]:
mapping = cudf.read_csv('./data/node_graph_map.csv')
mapping.head()

,node_id,graph_id
0,id000000F5-5180-4C03-B05D-B01352C54F89,0
1,id000003F8-9E09-4829-AD87-6DA4438D22D8,1
2,id000010DA-C89A-4198-847A-6E62815E038A,2
3,id000017A0-1843-4BC7-BCF7-C943B6780839,3
4,id00001B2A-155F-4CD3-8E06-7677ADC6AF74,4


We see that the `sssp` algorithm has put the `graph_id`s in the `vertex` column, so we will merge on that.

In [23]:
ex_dist.head()

,distance,vertex,predecessor
0,10449.427734,78912,288209
1,14686.548828,78913,3017223
2,16228.975586,78914,2678532
3,14788.810547,78915,2830870
4,15875.351562,78916,563879


In [24]:
road_nodes = road_nodes.merge(mapping, on='node_id')
road_nodes = road_nodes.merge(ex_dist, left_on='graph_id', right_on='vertex')
road_nodes.head()

,node_id,east,north,type,graph_id,distance,vertex,predecessor
0,id06742610-77C3-43B9-B644-CC2C3F7F46EA,389742.78125,403045.1875,pseudo node,77824,7297.651855,77824,684363
1,id067432FB-C36D-429C-B894-D3928802F8A7,579357.00000,286683.0000,road end,77825,13623.601562,77825,2255097
2,id06743603-CF7F-4C33-873C-D119A305D17E,313073.00000,372924.0000,junction,77826,10614.585938,77826,1627948
3,id06743886-7856-4C05-9286-07E486699FE5,327761.00000,196548.0000,junction,77827,16313.569336,77827,155553
4,id06743F0B-CD32-48DB-B08C-61A52C79A46B,336175.87500,399931.3750,road end,77828,8897.522461,77828,2435706


Next, we select those columns we are going to use for the visualization.

For color-scaling purposes, we get rid of the unreachable nodes with their extreme distances, and we invert the distance numbers so that brighter pixels indicate closer locations.

In [25]:
gdf = road_nodes[['east', 'north', 'distance']]
gdf = gdf[gdf['distance'] < 2**32]
gdf['distance'] = gdf['distance'].pow(1/2).mul(-1)

Otherwise, this visualization will be largely similar to the scatterplots we made to visualize the population, but instead of coloring by point density as in those cases, we will color by mean travel time to the nodes within a pixel.

In [26]:
cxf_data = cxf.DataFrame.from_dataframe(gdf)

In [27]:
chart_width = 600
heatmap_chart = cxf.charts.datashader.scatter(x='east', y='north', 
                                              width=chart_width, 
                                              height=int((gdf['north'].max() - gdf['north'].min()) / 
                                                         (gdf['east'].max() - gdf['east'].min()) *
                                                          chart_width),
                                              #palettes=Plasma256, # try also Turbo256, Viridis256, Magma
                                              #pixel_shade_type='linear', # can also be log, cbrt
                                              aggregate_col='distance',
                                              aggregate_fn='mean',
                                              #point_shape='square',
                                              point_size=1)

In [ ]:
dash = cxf_data.dashboard([heatmap_chart], theme=cxf.themes.dark, data_size_widget=True)

heatmap_chart.view()

In [ ]:
%%js
var host = window.location.host;
element.innerText = "'"+host+"'";

Set `my_url` in the next cell to the value just printed, making sure to include the quotes:

In [ ]:
my_url = # TODO: Set this value to the print out of the cell above, including the quotes.
dash.show(my_url, port=8789)

... and you can run the next cell to generate a link to the dashboard:

In [ ]:
%%js
var host = window.location.host;
var url = 'http://'+host+'/lab/proxy/8789/';
element.innerHTML = '<a style="color:blue;" target="_blank" href='+url+'>Open Dashboard</a>';

In [ ]:
dash.stop()

## Next

This concludes the second section of the workshop. In the third section, you'll put the skills you've learned in this workshop to the test by helping over several simulated days to address a national epidemic.

<br>
<div align="center"><h2>Optional: Restart the Kernel</h2></div>

If you plan to do additional work in other notebookes, please restart the kernel:

In [ ]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)